Now I'm trying to play with pokemon dataset, and see what I can do with it..

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.models import *
from keras.layers import *
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/pokemon/Pokemon.csv")

In [ ]:
df

let's inspect the data first, Let's see the Type distribution

In [ ]:
ax = sns.countplot(df["Type 1"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

In [ ]:
ax = sns.countplot(df["Type 2"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

Now, I'm gonna convert string features to integer

In [ ]:
type1list = list(set(df["Type 1"]))
type1 = df["Type 1"]
type1 = [type1list.index(x) for x in type1]
df["Type 1"] = type1
df

In [ ]:
type2list = list(set(df["Type 2"]))
type2 = df["Type 2"]
type2 = [type2list.index(x) for x in type2]
df["Type 2"] = type2
df

In [ ]:
Legendary = df["Legendary"]
Legendary = [int(x==True) for x in Legendary]
df["Legendary"] = Legendary
df

In [ ]:
df

In [ ]:
model.summary()

Now, I'll see the correlation between features

In [ ]:
plt.figure(figsize=(16,9))
sns.heatmap(df.corr(), annot = True)

It seems that the Legendary feature have big correlation with the other features, so I'll try to predict whether a pokemon is Legendary or not

I'll drop the index and name features, and use "Legendary" feature as the label

In [ ]:
X = df.drop(["Name", "Legendary", "#"], axis=1)
Y = df["Legendary"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [ ]:
X_train

For this time, i'm using f1-score

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

gamma = 2.0
epsilon = K.epsilon()
def focal_loss(y_true, y_pred):
    pt = y_pred * y_true + (1-y_pred) * (1-y_true)
    pt = K.clip(pt, epsilon, 1-epsilon)
    CE = -K.log(pt)
    FL = K.pow(1-pt, gamma) * CE
    loss = K.sum(FL, axis=1)
    return loss

I'll predict using neural network

In [ ]:
model = Sequential()

model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(optimizer = Adam(1e-4),
             loss = "binary_crossentropy",
             metrics = ["accuracy", f1_m]
)

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
early_stop = EarlyStopping(patience=10, verbose=1, monitor='f1_m', mode='max')
# checkpoint = ModelCheckpoint(os.path.join(model_save_path, model.name+".h5"), save_weights_only=True,
#                              verbose=1, save_best_only=True, monitor='f1_m', mode='max')

In [ ]:
history = model.fit(
        X_train,
        y_train,
        epochs = 80,
        callbacks=[early_stop],
        verbose = 1
    
)

Prediction

In [ ]:
pred = model.predict(X_test)
for x in range(len(y_test)):
    print(pred[x], int(pred[x] > 0.5), list(y_test)[x])
    
    
res = [int(int(pred[x] > 0.5) == list(y_test)[x]) for x in range(len(y_test))]
# res

Now check the accuracy

In [ ]:
print(f"accuracy: {(np.sum(res)/len(res))*100}%")

**CONCLUSION**
I'm not sure if this model works well, since the data is very imbalanced

I'll figure something out tomorrow